In [ ]:
import os
os.chdir('..')

In [ ]:
from datetime import timedelta
import time

import polars as pl

from generate.kline import merge_kline_lazy, detect_gaps_lazy
from config import TradeType, BINANCE_DATA_DIR
from util.log_kit import logger

pl.Config.set_tbl_rows(100)

In [ ]:
spot_results_dir = BINANCE_DATA_DIR / "results_data" / TradeType.spot / "klines" / "1m"
luna_files = list(spot_results_dir.glob('*LUNAUSDT*'))

In [ ]:
for f in luna_files:
    df = pl.read_parquet(f)
    print(f.stem, df["candle_begin_time"].is_sorted())
    display(df.head())
    display(df.tail())

In [ ]:
ufutures_results_dir = BINANCE_DATA_DIR / "results_data" / TradeType.um_futures / "klines" / "1m"
files = list(ufutures_results_dir.glob('*TLMUSDT*'))
for f in files:
    print(f.stem)

In [ ]:
for f in files:
    df = pl.read_parquet(f)
    print(f.stem, df["candle_begin_time"].is_sorted())
    display(df.head())
    display(df.tail())